## 第一部分：了解 nn.Module的基本操作

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [2]:
model = models.resnet18()

### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [4]:
for name, param in model.named_parameters():
    print(name, param.shape)

conv1.weight torch.Size([64, 3, 7, 7])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer2.0.conv1.weight torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight torch.Size([128])
layer2.0.bn1.bias torch.Size([128])
layer2.0.conv2.weight torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight torch.Size([128])
layer2.0.bn2.bias torch.Size([128])
layer2.0.downsample.0.weight torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight torch.Size([128])
layer2.0.downsample.1.bias torch.Size([128])
layer2.1.conv1.weight tor

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [5]:
input_ = torch.randn(1, 3, 128, 128)

In [6]:
for param in model.parameters():
    param.requires_grad = False

In [7]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [93]:
input_ = torch.randn(10,12)
### 
modules = []
modules.append(nn.Linear(12,10))
modules.append(nn.BatchNorm1d(10))
layer = nn.Sequential(*modules)    
    
mod2 = []
mod2.append(nn.Linear(10,10))
mod2.append(nn.BatchNorm1d(10)) 
layer_ = nn.Sequential(*mod2)
    
re_layer = nn.ModuleList()
for i in range(10):
    re_layer.append(layer_)

In [126]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.sequential = layer
        self.repeat_linear = re_layer
        self.output = nn.Linear(10,3)
        
    def forward(self, x):
        output = self.sequential(x)
        for _,module in enumerate(self.repeat_linear):
            output = module(output)
        x = self.output(output)
        return x

In [127]:
model = Model()

In [128]:
model

Model(
  (sequential): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (repeat_linear): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): Sequential(
      (0): Linear(in_features=10, out_fe

In [137]:
input_ = torch.randn(10,12)
outupt = model(input_)

In [138]:
outupt 

tensor([[-0.1847,  0.0098,  0.0652],
        [-0.2136, -0.6541,  0.3158],
        [ 0.0778, -0.3611,  0.2286],
        [-0.2750, -0.0297, -0.4409],
        [ 0.4408, -0.0394,  0.0486],
        [ 1.0801,  0.1891, -0.0407],
        [-0.7478, -0.1051, -0.5470],
        [-0.4129,  0.3122, -0.2351],
        [-1.0473, -0.0440, -0.2280],
        [ 0.3340, -0.2722, -0.1211]], grad_fn=<AddmmBackward>)